# API file formats



```{yaml}
type: file # <1>
description: "A normalised data with a PCA embedding, HVG selection and a kNN graph" # <2>
example: "dataset.h5ad"  # <3>
info:  # <4>
  label: "Dataset+PCA+HVG+kNN"
  slots:
    # 

```

1. Type of the entry. For file speicifications, it is always `file`
2. Description of the file, useful for quickly understanding what the file contains
3. Example of the file name in the command line
4. Info


### Example

```{yaml}
type: file
description: "A normalised data with a PCA embedding, HVG selection and a kNN graph"
example: "dataset.h5ad"
info:
  label: "Dataset+PCA+HVG+kNN"
  slots:
    layers: 
      - type: integer
        name: counts
        description: Raw counts
        required: true
      - type: double
        name: normalized
        description: Normalised expression values
    obs:
      - type: string
        name: celltype
        description: Cell type information
        required: false
    var:
      - type: boolean
        name: hvg
        description: Whether or not the feature is considered to be a 'highly variable gene'
        required: true
    uns:
      - type: string
        name: dataset_id
        description: "A unique identifier for the dataset"
        required: true
```

# API component specs


{{< include ../_blocks/_clone_repo.qmd >}}
{{< include /_include/_synchonize_tabs.qmd >}}

Components are the most modular building blocks of the overall task.
Each method, metric or data processing workflow should have its own component.
This way, components can be separated arranged into end-to-end workflows.

For different variants of the same task, such as multiple different methods or metrics, we can define a common API so that the further processing of the outputs can be kept the same across methods or metrics.
It is good practice to formalise the common metadata of all methods or metrics in a generic API specification.

## Specification

madtory vs optional vs extendable

<!-- TODO: generate table with different  -->

```{yaml}
functionality:  # <1>
  namespace: <task_name>/<subtask>
  info:
    type: method
    output_type: embedding
  arguments:
    - name: --input
      __merge__: anndata_unintegrated.yaml
    - name: --output
      direction: output
      __merge__: anndata_integrated_embedding.yaml
    - name: --hvg
      type: boolean
      description: Whether to subset to highly variable genes
      default: false
      required: false
  test_resources:
    - path: ../../../../resources_test/batch_integration/pancreas/
    - type: python_script
      path: generic_test.py
```

### Example

```{yaml}
functionality:
  namespace: <task_name>/<subtask>
  info:
    type: method
    output_type: embedding
  arguments:
    - name: --input
      __merge__: anndata_unintegrated.yaml
    - name: --output
      direction: output
      __merge__: anndata_integrated_embedding.yaml
    - name: --hvg
      type: boolean
      description: Whether to subset to highly variable genes
      default: false
      required: false
  test_resources:
    - path: ../../../../resources_test/batch_integration/pancreas/
    - type: python_script
      path: generic_test.py
```


## Applying API to specific component

```{yaml}
__merge__: ../../api/comp_method_embedding.yaml
functionality:
  name: scanorama_embed
  description: "Efficient integration of heterogeneous single-cell 
    transcriptomes using Scanorama"
  info:
    method_name: Scanorama
    paper_reference: "hie2019efficient"
    code_url: https://github.com/brianhie/scanorama
    v1_url: openproblems/tasks/_batch_integration/batch_integration_graph/methods/scanorama.py
    v1_commit: 29803b95c88b4ec5921df2eec7111fd5d1a95daf
    preferred_normalization: log_cpm
    variants:
      scanorama_embed_full_unscaled:
      scanorama_embed_hvg_unscaled:
        hvg: true
      scanorama_embed_hvg_scaled:
        hvg: true
        preferred_normalization: log_cpm_scaled
      scanorama_embed_full_scaled:
        preferred_normalization: log_cpm_scaled
  resources:
    - type: python_script
      path: script.py
platforms:
  - type: docker
    image: mumichae/scib-base:1.1.3
    setup:
      - type: python
        pypi:
          - scanorama
          - pyyaml
  - type: nextflow
```